In [1]:
# Importing libraries
import os
import numpy as np
from pathlib import Path
import rasterio as rio
from rasterio.merge import merge

In [2]:
# path to separate files
load_path = Path("../data/mask")

In [3]:
os.listdir(load_path)

['.DS_Store', 'mask1.tif', 'mask2.tif', 'merged']

In [4]:
def merge_files(input_path: Path, output_name: str):
    """For GFM products which are not 1 piece
    Input:Path -> Gets as input path of the files to be merged
    Output:Path -> Merges the files, saves to ./merged/ folder
    and returns the output file's path
    """

    # defining output file's path
    output_path = Path(os.path.join(input_path, "merged"))
    output_path.mkdir(parents=True, exist_ok=True)
    output_file = Path(os.path.join(output_path, output_name))

    # break if the file already exists
    if os.path.exists(output_file):
        return output_file

    # list of *.tif files paths in given directory
    files_path = os.listdir(input_path)
    files_path = filter(lambda x: x.endswith(".tif"), files_path)
    files_path = list(files_path)

    # loading files
    files = []
    for f in files_path:
        temp_file_path = os.path.join(input_path, f)
        file = rio.open(temp_file_path)
        files.append(file)

    # merging files
    merged, output_trans = merge(files)

    # new metadata
    output_meta = file.meta.copy()
    output_meta.update(
        {
            "driver": "GTiff",
            "height": merged.shape[1],
            "width": merged.shape[2],
            "transform": output_trans,
        }
    )

    # saving merged file into disk
    with rio.open(output_file, "w", **output_meta) as m:
        m.write(merged)

    return output_file

In [5]:
merge_files(input_path=load_path, output_name="mask.tif")

PosixPath('../data/mask/merged/mask.tif')